In [1]:
#!/usr/bin/python3.6

"""Code to format info on movies in a csv file for publication.
Created by Doug Ray and last updated 07/30/2019."""

# built-in libraries
import csv
from collections import OrderedDict, defaultdict
import datetime
from email.message import EmailMessage
import os.path
import re
import smtplib
import sqlite3
import sys

# installed with pip
import numpy as np
import pandas as pd

# local modules
import creds

# Create list of dictionaries for each row.
imacscrapedir = '/Users/rayd/workspace/cinema/scrapes/'
airscrapedir = '/Users/Doug/workspace/cinema/scrapes/'
today = datetime.date.today()
scrapedate = str(today.strftime("%Y-%m-%d"))
scrapefile = "fandango-" + scrapedate + ".csv"
scrapefile = "fandango-2019-07-11.csv"
scrapefile = "cinemascrape.csv"
scrape_path = imacscrapedir + scrapefile

listings = []

with open(scrape_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        layout_dct = {
            'addy': row[0],
            'theater': row[1],
            'title': row[2],
            'mpaa': row[3],
            'stars': row[4],
            'times': row[5],
            'format': row[6],
            'synopsis': row[7],
            'date': row[8],
            'city': row[9]
        }
        listings.append(layout_dct)
        
del listings[0] #remove header row   
      
# Create Pandas dataframe for listings
df = pd.DataFrame(listings)
# fix up titles, times and days, keep just needed columns, sort rows by movie+day
df.title = df.title.apply(lambda x: '"' + str(x) + '"')
df.times = df.times.str.replace(':00', '')
df.times = df.times.str.replace('|', ',')
df.times = df.times.str.replace('p', ' p.m.')
df.times = df.times.str.replace(' p.m. , ', ', ')
df.times = df.times.str.replace('a', ' a.m.,')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['movieday'] = df['date'].dt.weekday_name
df = df.drop('city', axis=1)
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.drop('date', axis=1)
df['screen'] = df['title'].str.cat(df['format'],sep=", ")
df.screen = df.screen.str.cat(df['mpaa'],sep=", ")
df = df[['theater', 'screen', 'movieday', 'times']]
df.sort_values(["screen", "movieday"], axis =0,
              ascending=True, inplace=True)
df = df.drop_duplicates()

# Ocala market
oc6_df = df.loc[df['theater'].str.contains('Ocala Center 6')]
bv_df = df.loc[df['theater'].str.contains('Belleview')]
mt_df = df.loc[df['theater'].str.contains('Marion')]
omp_df = df.loc[df['theater'].str.contains('Old Mill')]
regaloca_df = df.loc[df['theater'].str.contains('Regal Hollywood')]
rialto_df = df.loc[df['theater'].str.contains('Rialto')]
amc_df = df.loc[df['theater'].str.contains('AMC Lake')]
barn_df = df.loc[df['theater'].str.contains('Barnstorm')]

# Gainesville market
celeb_df = df.loc[df['theater'].str.contains('Celebration')]
hipp_df = df.loc[df['theater'].str.contains('Hippodrome')]
butler_df = df.loc[df['theater'].str.contains('Butler')]
regalgnv_df = df.loc[df['theater'].str.contains('Regal Royal Park')]
uac_df = df.loc[df['theater'].str.contains('Regal UA Cinema 90')]

# Convert each to list of ordered dicts
# NOTE: Likely a more pythonic way to do this exists
mt_dict = mt_df.to_dict(into=OrderedDict)
mt_dlist = list(mt_dict.items())
amc_dict = amc_df.to_dict(into=OrderedDict)
amc_dlist = list(amc_dict.items())
barm_dict = barn_df.to_dict(into=OrderedDict)
barn_dlist = list(barm_dict.items())
bv_dict = bv_df.to_dict(into=OrderedDict)
bv_dlist = list(bv_dict.items())
oc6_dict = oc6_df.to_dict(into=OrderedDict)
oc6_dlist = list(oc6_dict.items())
omp_dict = omp_df.to_dict(into=OrderedDict)
omp_dlist = list(omp_dict.items())
regaloca_dict = regaloca_df.to_dict(into=OrderedDict)
regaloca_dlist = list(regaloca_dict.items())
rialto_dict = rialto_df.to_dict(into=OrderedDict)
rialto_dlist = list(rialto_dict.items())
celeb_dict = celeb_df.to_dict(into=OrderedDict)
celeb_dlist = list(celeb_dict.items())
hipp_dict = hipp_df.to_dict(into=OrderedDict)
hipp_dlist = list(hipp_dict.items())
butler_dict = butler_df.to_dict(into=OrderedDict)
butler_dlist = list(butler_dict.items())
regalgnv_dict = regalgnv_df.to_dict(into=OrderedDict)
regalgnv_dlist = list(regalgnv_dict.items())
uac_dict = uac_df.to_dict(into=OrderedDict)
uac_dlist = list(uac_dict.items())

oca_dlists = [mt_dlist, amc_dlist, barn_dlist, bv_dlist, oc6_dlist,
              omp_dlist, regaloca_dlist, rialto_dlist]
gnv_dlists = [hipp_dlist, butler_dlist, celeb_dlist, regalgnv_dlist,
             uac_dlist]

In [2]:
def fridays(dlist):
    
    """Logic to group values from dicts into a narrative for Friday listings"""
    
    global pn
    pn = ""
    
    if 'AMC Lake Square 12' in list(dlist[0][1].values()):
        pn += 'AMC Lake Square 12, 10401-015 US-441, Leesburg\n'
    elif 'Barnstorm Theater' in list(dlist[0][1].values()):
        pn += '\nBarnstorm Theater, 2720 Brownwood Blvd., The Villages\n'
    elif 'Belleview Cinemas' in list(dlist[0][1].values()):
        pn += '\nBelleview Cinemas, 10845 SE US Highway 441, Belleview\n'
    elif 'Hippodrome State Theatre' in list(dlist[0][1].values()):
        pn += '\nHippodrome State Theatre, 25 SE 2nd Place, Gainesville\n'
    elif 'Marion Theatre' in list(dlist[0][1].values()):
        pn += '\nMarion Theatre, 50 S. Magnolia Ave., Ocala\n'
    elif 'Ocala Center 6' in list(dlist[0][1].values()):
        pn += '\nOcala Center 6, 2021 Silver Springs Blvd, Ocala\n'
    elif 'Old Mill Playhouse' in list(dlist[0][1].values()):
        pn += '\nOld Mill Playhouse, 1000 Old Mill Run, The Villages\n'
    elif 'Regal Butler Town Center' in list(dlist[0][1].values()):
        pn += '\nRegal Butler Town Center, 3101 SW 35th Blvd., Gainesville\n'
    elif 'Regal Celebration Pointe & RPX' in list(dlist[0][1].values()):
        pn += '\nRegal Celebration Pointe & RPX, 4901 SW 31st Place, Gainesville\n'
    elif 'Regal Hollywood & IMAX - Ocala' in list(dlist[0][1].values()):
        pn += '\nRegal Hollywood & IMAX-Ocala, 2801 SW 27th Ave., Ocala\n'
    elif 'Regal Royal Park' in list(dlist[0][1].values()):
        pn += '\nRegal Royal Park, 3702 Newberry Road, Gainesville\n'
    elif 'Regal UA Cinema 90' in list(dlist[0][1].values()):
        pn += '\nRegal UA Cinema 90, 2830 Hwy. 90 West, Lake City\n'
    elif 'Rialto Theatre Spanish Springs Town Square' in list(dlist[0][1].values()):
        pn += '\nRialto Theatre Spanish Springs Town Square\n'
                
    #Convert ordered dicts to lists of movies, days and times
    movievals = list(dlist[1][1].values())
    dayvals = list(dlist[2][1].values())
    timevals = list(dlist[3][1].values())
    
    while len(movievals) >= 1:
        if len(movievals) >= 3:
            # same movie all three days; this will be almost all films
            if movievals[0] == movievals[1] and movievals[1] == movievals[2]: # same movie all three days
                # must be Friday-Sunday run
                screening = str(movievals[0])
                n = 3 # how many items to remove from lists before loop runs again
                fri = str(timevals[0])
                sat = str(timevals[1])
                sun = str(timevals[2])
                if dayvals[0] == 'Friday' and dayvals[1] == 'Saturday' and dayvals[2] == 'Sunday':
                    if fri == sat and fri == timevals[2]: # want all common
                        pn += f"{screening}, today-Sunday at {fri}\n"
                    elif fri != sat and fri != sun and sat == sun: # want fri unique
                        pn += f"{screening} today at {fri}, Saturday-Sunday at {sat}\n"
                    elif sat != fri and sat != sun and fri == sun: # want sat unique
                        pn += f"{screening}, today and Sunday at {fri}, Saturday at {sat}\n"
                    elif sun != fri and sun != sat and fri == sat: # want sun unique
                        pn += f"{screening}, today and Saturday at {fri}, Sunday at {sun}\n"
                    elif fri != sat and fri != sun and sat !=sun: # want all unique
                        pn += f"{screening}, today at {fri}, Saturday at {sat}, Sunday at {sun}\n"
                    else:
                        pn += "Show times didn't work out!"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here"
                    break
            # movie shows just two days 
            elif movievals[0] == movievals[1] and movievals[1] != movievals[2]: 
                screening = str(movievals[0])
                n = 2
                if dayvals[0] == 'Friday' and dayvals[1] == 'Saturday':
                    fri = str(timevals[0])
                    sat = str(timevals[1])
                    if fri == sat:
                        pn += f"{screening}, today, Saturday at {fri}\n"
                    else:
                        pn += f"{screening}, today at {fri} Saturday at {sat}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Friday' and dayvals[1] == 'Sunday':
                    fri = str(timevals[0])
                    sun = str(timevals[1])
                    if fri == sun:
                        pn += f"{screening} today, Sunday at {fri}\n"
                    else:
                        pn += f"{screening} today at {fri}, Sunday at {sun}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday' and dayvals[1] == 'Sunday':
                    sat = str(timevals[0])
                    sun = str(timevals[1])
                    if sat == sun:
                        pn += f"{screening} Saturday, Sunday at {sat}\n"
                    else:
                        pn += f"{screening}, Saturday at {sat}, Sunday at {sun}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here"
                    break
            # it's a one-day show
            elif movievals[0] != movievals[1]:
                screening = str(movievals[0])
                n = 1
                if dayvals[0] == 'Friday':
                    fri = str(timevals[0])
                    pn += f"{screening}, today at {fri}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday':
                    sat = str(timevals[0])
                    pn += f"{screening}, Saturday at {sat}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Sunday':
                    sun = str(timevals[0])
                    pn += f"{screening}, Sunday at {sun}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here"
                    break
            else:
                pn += "A problem here"
                break
        elif len(movievals) >= 2: # We are down to the last one or two movies; should never happen
            # movie shows just two days 
            if movievals[0] == movievals[1]: 
                screening = str(movievals[0])
                n = 2
                if dayvals[0] == 'Friday' and dayvals[1] == 'Saturday':
                    fri = str(timevals[0])
                    sat = str(timevals[1])
                    if fri == sat:
                        pn += f"{screening}, today, Saturday at {fri}\n"
                    else:
                        pn += f"{screening}, today at {fri}, Saturday at {sat}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Friday' and dayvals[1] == 'Sunday':
                    fri = str(timevals[0])
                    sun = str(timevals[1])
                    if fri == sun:
                        pn += f"{screening} today, Sunday at {sun}\n"
                    else:
                        pn += f"{screening}, today at {fri}, Sunday at {sun}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday' and dayvals[1] == 'Sunday':
                    sat = str(timevals[0])
                    sun = str(timevals[1])
                    if sat == sun:
                        pn += f"{screening}, Saturday, Sunday at {sat}\n"
                    else:
                        pn += f"{screening}, Saturday at {sat}, Sunday at {sun}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here"
                    break
            # it's a one-day show
            elif movievals[0] != movievals[1]:
                screening = str(movievals[0])
                n = 1
                if dayvals[0] == 'Friday':
                    fri = str(timevals[0])
                    pn += f"{screening}, today at {fri}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday':
                    sat = str(timevals[0])
                    pn += f"{screening}, Saturday at {sat}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Sunday':
                    sun = str(timevals[0])
                    pn += f"{screening}, Sunday at {sun}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here"
                    break
        else:        
            n = 1
            if dayvals[0] == 'Friday':
                screening = str(movievals[0])
                fri = str(timevals[0])
                pn += f"{screening}, today at {fri}\n"
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]
            elif dayvals[0] == 'Saturday':
                screening = str(movievals[0])
                sat = str(timevals[0])
                pn += f"{screening}, Saturday at {sat}\n"
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]
            else:
                screening = str(movievals[0])
                sun = str(timevals[0])
                pn += f"{screening}, Sunday at {sun}\n"
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]
    return(pn)

In [3]:
def saturdays(dlist):
    
    global pn
    pn = ""
    
    """Logic to group values from dicts into a narrative for Saturday listings"""
    
    if 'AMC Lake Square 12' in list(dlist[0][1].values()):
        pn += 'AMC Lake Square 12, 10401-015 US-441, Leesburg\n'
    elif 'Barnstorm Theater' in list(dlist[0][1].values()):
        pn += '\nBarnstorm Theater, 2720 Brownwood Blvd., The Villages\n'
    elif 'Belleview Cinemas' in list(dlist[0][1].values()):
        pn += '\nBelleview Cinemas, 10845 SE US Highway 441, Belleview\n'
    elif 'Hippodrome State Theatre' in list(dlist[0][1].values()):
        pn += '\nHippodrome State Theatre, 25 SE 2nd Place, Gainesville\n'
    elif 'Marion Theatre' in list(dlist[0][1].values()):
        pn += '\nMarion Theatre, 50 S. Magnolia Ave., Ocala\n'
    elif 'Ocala Center 6' in list(dlist[0][1].values()):
        pn += '\nOcala Center 6, 2021 Silver Springs Blvd, Ocala\n'
    elif 'Old Mill Playhouse' in list(dlist[0][1].values()):
        pn += '\nOld Mill Playhouse, 1000 Old Mill Run, The Villages\n'
    elif 'Regal Butler Town Center' in list(dlist[0][1].values()):
        pn += '\nRegal Butler Town Center, 3101 SW 35th Blvd., Gainesville\n'
    elif 'Regal Celebration Pointe & RPX' in list(dlist[0][1].values()):
        pn += '\nRegal Celebration Pointe & RPX, 4901 SW 31st Place, Gainesville\n'
    elif 'Regal Hollywood & IMAX - Ocala' in list(dlist[0][1].values()):
        pn += '\nRegal Hollywood & IMAX-Ocala, 2801 SW 27th Ave., Ocala\n'
    elif 'Regal Royal Park' in list(dlist[0][1].values()):
        pn += '\nRegal Royal Park, 3702 Newberry Road, Gainesville\n'
    elif 'Regal UA Cinema 90' in list(dlist[0][1].values()):
        pn += '\nRegal UA Cinema 90, 2830 Hwy. 90 West, Lake City\n'
    elif 'Rialto Theatre Spanish Springs Town Square' in list(dlist[0][1].values()):
        pn += '\nRialto Theatre Spanish Springs Town Square\n'
                
    #Convert ordered dicts to lists of movies, days and times
    movievals = list(dlist[1][1].values())
    dayvals = list(dlist[2][1].values())
    timevals = list(dlist[3][1].values())

    while len(movievals) >= 1:
        if len(movievals) >= 3:
            # same movie all three days; this will be almost all films
            if movievals[0] == movievals[1] and movievals[1] == movievals[2]: # same movie all three days
                # must be Friday-Sunday run
                n = 3 # how many items to remove from lists before loop runs again
                screening = str(movievals[0])
                fri = str(timevals[0])
                sat = str(timevals[1])
                sun = str(timevals[2])
                if dayvals[0] == 'Friday' and dayvals[1] == 'Saturday' and dayvals[2] == 'Sunday':
                    if sat == sun:
                        pn += f"{screening}, today, Sunday at {sat}\n"
                    elif sat != sun:
                        pn += f"{screening}, today at {sat}, Sunday at {sun}\n"
                    else:
                        pn += "Show times didn't work out!"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here"
                    break
            # movie shows just two days 
            elif movievals[0] == movievals[1] and movievals[1] != movievals[2]: 
                screening = str(movievals[0])
                n = 2
                if dayvals[0] == 'Friday' and dayvals[1] == 'Saturday':
                    fri = str(timevals[0])
                    sat = str(timevals[1])
                    pn += f"{screening}, today at {sat}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Friday' and dayvals[1] == 'Sunday':
                    fri = str(timevals[0])
                    sun = str(timevals[1])
                    pn += f"{screening}, Sunday at {sun}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday' and dayvals[1] == 'Sunday':
                    sat = str(timevals[0])
                    sun = str(timevals[1])
                    if sat == sun:
                        pn += f"{screening}, today, Sunday at {sat}\n"
                    else:
                        pn += f"{screening}, today at {sat}, Sunday at {sun}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here"
                    break
            # it's a one-day show
            elif movievals[0] != movievals[1]:
                screening = str(movievals[0])
                n = 1
                if dayvals[0] == 'Saturday':
                    sat = str(timevals[0])
                    pn += f"{screening}, today at {sat}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                if dayvals[0] == 'Sunday':
                    sun = str(timevals[0])
                    pn += f"{screening}, Sunday at {sun}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]                    
                    continue
            else:
                pn += "We have a problem"
                break
        elif len(movievals) >= 2: # We are down to the last one or two movies; should never happen
            # movie shows just two days 
            if movievals[0] == movievals[1]: 
                screening = str(movievals[0])
                n = 2
                if dayvals[0] == 'Friday' and dayvals[1] == 'Saturday':
                    fri = str(timevals[0])
                    sat = str(timevals[1])
                    pn += f"{screening}, today at {sat}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Friday' and dayvals[1] == 'Sunday':
                    fri = str(timevals[0])
                    sun = str(timevals[1])
                    pn += f"{screening}, Sunday at {sun}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday' and dayvals[1] == 'Sunday':
                    sat = str(timevals[0])
                    sun = str(timevals[1])
                    if sat == sun:
                        pn += f"{screening}, today, Sunday at {sat}\n"
                    else:
                        pn += f"{screening}, today at {sat}, Sunday at {sun}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here"
                    break
            # it's a one-day show
            elif movievals[0] != movievals[1]:
                screening = str(movievals[0])
                n = 1
                if dayvals[0] == 'Saturday':
                    sat = str(timevals[0])
                    pn += f"{screening}, today at {sat}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Sunday':
                    sun = str(timevals[0])
                    pn += screening + ", Sunday at " + sun + "\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here"
                    break
        else:        
            n = 1
            if dayvals[0] == 'Saturday':
                screening = str(movievals[0])
                sat = str(timevals[0])
                pn += f"{screening}, today at {sat}\n"
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]
            else:
                screening = str(movievals[0])
                sun = str(timevals[0])             
                pn += f"{screening}, Sunday at {sun}\n"
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]
                
    return(pn)

In [4]:
def sundays(dlist):
    
    global pn
    pn = ""
    
    """Logic to group values from dicts into a narrative for Sunday listings"""
    
    if 'AMC Lake Square 12' in list(dlist[0][1].values()):
        pn += 'AMC Lake Square 12, 10401-015 US-441, Leesburg\n'
    elif 'Barnstorm Theater' in list(dlist[0][1].values()):
        pn += '\nBarnstorm Theater, 2720 Brownwood Blvd., The Villages\n'
    elif 'Belleview Cinemas' in list(dlist[0][1].values()):
        pn += '\nBelleview Cinemas, 10845 SE US Highway 441, Belleview\n'
    elif 'Hippodrome State Theatre' in list(dlist[0][1].values()):
        pn += '\nHippodrome State Theatre, 25 SE 2nd Place, Gainesville\n'
    elif 'Marion Theatre' in list(dlist[0][1].values()):
        pn += '\nMarion Theatre, 50 S. Magnolia Ave., Ocala\n'
    elif 'Ocala Center 6' in list(dlist[0][1].values()):
        pn += '\nOcala Center 6, 2021 Silver Springs Blvd, Ocala\n'
    elif 'Old Mill Playhouse' in list(dlist[0][1].values()):
        pn += '\nOld Mill Playhouse, 1000 Old Mill Run, The Villages\n'
    elif 'Regal Butler Town Center' in list(dlist[0][1].values()):
        pn += '\nRegal Butler Town Center, 3101 SW 35th Blvd., Gainesville\n'
    elif 'Regal Celebration Pointe & RPX' in list(dlist[0][1].values()):
        pn += '\nRegal Celebration Pointe & RPX, 4901 SW 31st Place, Gainesville\n'
    elif 'Regal Hollywood & IMAX - Ocala' in list(dlist[0][1].values()):
        pn += '\nRegal Hollywood & IMAX-Ocala, 2801 SW 27th Ave., Ocala\n'
    elif 'Regal Royal Park' in list(dlist[0][1].values()):
        pn += '\nRegal Royal Park, 3702 Newberry Road, Gainesville\n'
    elif 'Regal UA Cinema 90' in list(dlist[0][1].values()):
        pn += '\nRegal UA Cinema 90, 2830 Hwy. 90 West, Lake City\n'
    elif 'Rialto Theatre Spanish Springs Town Square' in list(dlist[0][1].values()):
        pn += '\nRialto Theatre Spanish Springs Town Square\n'
                
    #Convert ordered dicts to lists of movies, days and times
    movievals = list(dlist[1][1].values())
    dayvals = list(dlist[2][1].values())
    timevals = list(dlist[3][1].values())
    screening = str(movievals[0]) # for f-strings

    while len(movievals) >= 1:
        if len(movievals) >= 3:
            # same movie all three days; this will be almost all films
            if movievals[0] == movievals[1] and movievals[1] == movievals[2]: # same movie all three days
                # must be Friday-Sunday run
                screening = str(movievals[0])
                n = 3 # how many items to remove from lists before loop runs again
                sun = str(timevals[2])
                if dayvals[0] == 'Friday' and dayvals[1] == 'Saturday' and dayvals[2] == 'Sunday':
                    pn += f"{screening}, today at {sun}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    pn += "A problem here"
                    break
            # movie shows just two days 
            elif movievals[0] == movievals[1] and movievals[1] != movievals[2]: 
                screening = str(movievals[0])
                n = 2
                if dayvals[0] == 'Friday' and dayvals[1] == 'Sunday':
                    sun = str(timevals[1])
                    pn += f"{screening}, today at {sun}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday' and dayvals[1] == 'Sunday':
                    sun = str(timevals[1])
                    pn += f"{screening}, today at {sun}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
            # it's a one-day show
            else:
                screening = str(movievals[0])
                sun = str(timevals[0])
                n = 1
                if dayvals[0] == 'Sunday':
                    pn += f"{screening}, today at {sun}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]                    
                    continue
        elif len(movievals) >= 2: # We are down to the last one or two movies; should never happen
            # movie shows just two days 
            if movievals[0] == movievals[1]: 
                screening = str(movievals[0])
                n = 2
                if dayvals[0] == 'Friday' and dayvals[1] == 'Sunday':
                    sun = str(timevals[1])
                    pn += f"{screening}, today at {sun}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                elif dayvals[0] == 'Saturday' and dayvals[1] == 'Sunday':
                    sun = str(timevals[1])
                    pn += f"{screening}, today at {sun}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
            # it's a one-day show
            elif movievals[0] != movievals[1]:
                screening = str(movievals[0])
                sun = str(timevals[0])
                n = 1
                if dayvals[0] == 'Sunday':
                    pn += f"{screening}, today at {sun}\n"
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
                else:
                    del movievals[:n]
                    del dayvals[:n]
                    del timevals[:n]
                    continue
        else:        
            n = 1
            if dayvals[0] == 'Sunday':
                screening = str(movievals[0])
                sun = str(timevals[0])
                pn += f"{screening}, today at {sun}\n"
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]
            else:
                screening = str(movievals[0])
                del movievals[:n]
                del dayvals[:n]
                del timevals[:n]
                continue
    return pn


In [10]:
sender = 'data@sunwriters.com'
receiver = 'doug.ray@starbanner.com'
gmail_pass = creds.gmail_pass

if __name__ == "__main__":
    
    today = datetime.date.today()
    weekday = today.weekday()

    if (weekday == 3): # Thursday
        # The new movie list files:
        #path_directory = os.path.dirname(os.path.abspath(__file__))
        path_directory = ''
        ocalafri = os.path.join(path_directory, 'ocalafriday.txt')
        ocalasat = os.path.join(path_directory, 'ocalasaturday.txt')
        ocalasun = os.path.join(path_directory, 'ocalasunday.txt')
        gnvfri = os.path.join(path_directory, 'gnvfriday.txt')
        gnvsat = os.path.join(path_directory, 'gnvaturday.txt')
        gnvsun = os.path.join(path_directory, 'gnvsunday.txt')
        
        if os.path.exists(ocalafri):
            os.remove(ocalafri)
        else:
            print(f"The old {ocalafri} isn't there.")
            
        if os.path.exists(ocalasat):
            os.remove(ocalasat)
        else:
            print(f"The old {ocalasat} isn't there.")
            
        if os.path.exists(ocalasun):
            os.remove(ocalasun)
        else:
            print(f"The old {ocalasun} isn't there.")
            
        if os.path.exists(gnvfri):
            os.remove(gnvfri)
        else:
            print(f"The old {gnvfri} isn't there.")
            
        if os.path.exists(gnvsat):
            os.remove(gnvsat)
        else:
            print(f"The old {gnvsat} isn't there.")
            
        if os.path.exists(gnvsun):
            os.remove(gnvsun)
        else:
            print(f"The old {gnvsun} isn't there.")
            
        # Write into to indicate in email which list is received.
        
        with open(ocalafri, 'w') as f:
            f.write(f"\nThese are listings for Ocala, Friday publication\n")
            
        with open(ocalasat, 'w') as f:
            f.write(f"\nThese are listings for Ocala, Saturday publication\n")
        
        with open(ocalasun, 'w') as f:
            f.write(f"\nThese are listings for Ocala, Sunday publication\n")
        
        with open(gnvfri, 'w') as f:
            f.write(f"\nThese are listings for Gainesville, Friday publication\n")
            
        with open(gnvsat, 'w') as f:
            f.write(f"\nThese are listings for Gainesville, Saturday publication\n")
            
        with open(gnvsun, 'w') as f:
            f.write(f"\nThese are listings for Gainesville, Sunday publication\n")
            
        # Now run the functions
        for dlist in oca_dlists:
            fridays(dlist)
            with open(ocalafri, 'a+') as f:
                f.write(pn)
                
            saturdays(dlist)
            with open(ocalasat, 'a+') as f:
                f.write(pn)
                
            sundays(dlist)
            with open(ocalasun, 'a+') as f:
                f.write(pn)
                
        for dlist in gnv_dlists:
            fridays(dlist)
            with open(gnvfri, 'a+') as f:
                f.write(pn)
            
            saturdays(dlist)
            with open(gnvsat, 'a+') as f:
                f.write(pn)
                
            sundays(dlist)
            with open(gnvsun, 'a+') as f:
                f.write(pn)
                
        # Send emails
        mlists = [ocalafri, ocalasat, ocalasun, gnvfri, gnvsat, gnvsun]

        for mlist in mlists:
            with open(mlist) as fp:
                msg = EmailMessage()
                msg.set_content(fp.read())

            sender = 'data@sunwriters.com'
            gmail_pass = '%WatchingTheDetectives'

            msg['from'] = 'data@sunwriters.com'
            msg['To'] = 'doug.ray@starbanner.com'
            msg['Subject'] = 'Movie listing for publication'

            try:
                server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
                server.ehlo()
                server.login(sender, gmail_pass)
                server.send_message(msg)
                server.quit()
                print('Test message sent!')
            except:
                print('Something went wrong.')        

    else:
        print("Today's not Thursday! No lists for you.")


Test message sent!
Test message sent!
Test message sent!
Test message sent!
Test message sent!
Test message sent!
